# 1.1

In [ ]:
%{
#include <stdio.h>
%}

%%
.       ECHO;
\n      ECHO;
%%

int yywrap(void) {
    return 1;
}

int main(void) {
    yylex();
    return 0;
}

# 1.2

In [ ]:
%{
#include <stdio.h>
%}

%%
[ \t\n]+     ;            // Match and skip all white spaces
.            ECHO;        // Print all other characters
%%

int yywrap(void) {
    return 1;
}

int main(void) {
    yylex();
    return 0;
}

# 2.1

In [ ]:
%{
#include <stdio.h>
%}

%%
// Keywords
"int"|"char"|"for"|"if"|"while"|"then"|"return"|"do" {printf("Keyword: %s\n", yytext);}

// Operators
[+\-*/%] {printf("Operator: %s\n", yytext);}

// Special Symbols
[(){};] {printf("Special Symbol: %s\n", yytext);}

// Constants (integers)
[0-9]+ {printf("Constant: %s\n", yytext);}

// Valid Identifiers
[a-zA-Z_][a-zA-Z0-9_]* {printf("Valid Identifier: %s\n", yytext);}

// Invalid Characters (optional)
[^a-zA-Z0-9_ \t\n] {printf("Invalid Character: %s\n", yytext);}
%%

int yywrap() {
    return 1;
}

int main() {
    yylex();
    return 0;
}


# 2.2

In [ ]:
%{
#include <stdio.h>
%}

%%
// Ignoring spaces, tabs, and newlines
[ \t\n]+                { /* Do nothing (ignore whitespace) */ }

// Keywords
"int"|"float"|"char"|"if"|"else"|"while"|"return" {
    printf("Keyword: %s\n", yytext);
}

// Identifiers
[a-zA-Z_][a-zA-Z0-9_]*  { printf("Identifier: %s\n", yytext); }

// Numbers
[0-9]+                  { printf("Number: %s\n", yytext); }

// Operators
[+\-*/=<>]              { printf("Operator: %s\n", yytext); }

// Special Symbols
[(){};,]                { printf("Special Symbol: %s\n", yytext); }

// Invalid Characters
.                       { printf("Invalid Character: %s\n", yytext); }

%%

int yywrap() {
    return 1;
}

int main() {
    yylex();
    return 0;
}


# 3.1

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <ctype.h>

int n, m = 0;
char a[10][10], f[10];

void first(char c);
void follow(char c);

void follow(char c) {
    if (a[0][0] == c) {
        f[m++] = '$';
    }
    for (int i = 0; i < n; i++) {
        for (int j = 2; j < strlen(a[i]); j++) {
            if (a[i][j] == c) {
                if (a[i][j + 1] != '\0') {
                    first(a[i][j + 1]);
                }
                if (a[i][j + 1] == '\0' && c != a[i][0]) {
                    follow(a[i][0]);
                }
            }
        }
    }
}

void first(char c) {
    if (!isupper(c)) {
        f[m++] = c;
        return;
    }
    for (int k = 0; k < n; k++) {
        if (a[k][0] == c) {
            if (a[k][2] == '$') {
                follow(a[k][0]);
            } else if (!isupper(a[k][2])) {
                f[m++] = a[k][2];
            } else {
                first(a[k][2]);
            }
        }
    }
}

int main() {
    int z;
    char c;
    printf("Enter the number of productions: ");
    scanf("%d", &n);
    
    printf("Enter the productions (ε = $):\n");
    for (int i = 0; i < n; i++) {
        scanf("%s", a[i]);
    }
    
    do {
        m = 0;
        printf("Enter the element whose FIRST & FOLLOW is to be found: ");
        scanf(" %c", &c);
        
        first(c);
        printf("FIRST(%c) = { ", c);
        for (int i = 0; i < m; i++) {
            printf("%c ", f[i]);
        }
        printf("}\n");
        
        m = 0; // Reset for FOLLOW
        follow(c);
        printf("FOLLOW(%c) = { ", c);
        for (int i = 0; i < m; i++) {
            printf("%c ", f[i]);
        }
        printf("}\n");
        
        printf("Do you want to continue (0/1)? ");
        scanf("%d", &z);
    } while (z == 1);
    
    return 0;
}


# 3.2

In [ ]:
%{
#include <stdio.h>
%}

%%
// Ignoring redundant whitespace
[ \t\n]+                     { /* Ignored */ }

// Keywords
"int"|"float"|"char"|"if"|"else"|"while"|"return" {
    printf("Keyword: %s\n", yytext);
}

// Identifiers
[a-zA-Z_][a-zA-Z0-9_]*       { printf("Identifier: %s\n", yytext); }

// Numbers
[0-9]+                       { printf("Number: %s\n", yytext); }

// Operators
[+\-*/=<>]                   { printf("Operator: %s\n", yytext); }

// Special symbols
[(){};,]                     { printf("Special Symbol: %s\n", yytext); }

// Invalid characters
.                            { printf("Invalid Character: %s\n", yytext); }

%%

int yywrap() {
    return 1;
}

int main() {
    yylex();
    return 0;
}


# 4.1

In [ ]:
#include <stdio.h>
#include <string.h>

char stack[20];
int top = -1;

void push(char item) {
    if (top >= 19) {
        printf("STACK OVERFLOW\n");
        return;
    }
    stack[++top] = item;
    printf("Push -- %c\n", item);
}

char pop() {
    if (top == -1) {
        printf("STACK UNDERFLOW\n");
        return '\0';
    }
    char c = stack[top--];
    printf("Pop -- %c\n", c);
    return c;
}

char TOS() {
    return stack[top];
}

int convert(char item) {
    switch (item) {
        case 'i': return 0;
        case '+': return 1;
        case '*': return 2;
        case '$': return 3;
        default : return -1;
    }
}

int main() {
    // Operator precedence table
    //       i    +    *    $
    char pt[4][4] = {
        /* i */ {'-', '>', '>', '>'},
        /* + */ {'<', '>', '<', '>'},
        /* * */ {'<', '>', '>', '>'},
        /* $ */ {'<', '<', '<', '1'}
    };

    char input[20];
    int ptr = 0;

    printf("Enter input ending with $: ");
    scanf("%s", input);

    push('$');

    while (ptr <= strlen(input)) {
        if (TOS() == '$' && input[ptr] == '$') {
            printf("SUCCESS\n");
            return 0;
        }

        int row = convert(TOS());
        int col = convert(input[ptr]);

        if (row == -1 || col == -1) {
            printf("Invalid character in input\n");
            return 1;
        }

        char relation = pt[row][col];

        if (relation == '<') {
            push(input[ptr]);
            ptr++;
        } else if (relation == '>') {
            pop();
        } else if (relation == '-') {
            printf("Invalid move\n");
            return 1;
        } else if (relation == '1') {
            printf("SUCCESS\n");
            return 0;
        }
    }

    return 0;
}


# 4.2

In [ ]:
#include<stdio.h>
#include<ctype.h>
#include<string.h>

void Tp(); 
void Ep(); 
void E(); 
void T(); 
void check(); 

int count, flag; 
char expr[100]; 

int main() {
    count = 0;
    flag = 0;

    printf("Enter an Algebraic Expression: ");
    scanf("%s", expr);

    E();  // Start parsing from E

    if ((strlen(expr) == count) && (flag == 0))
        printf("The expression %s is valid\n", expr);
    else
        printf("The expression %s is invalid\n", expr);

    return 0;
}

// E → T Ep
void E() {
    T();
    Ep();
}

// Ep → + T Ep | ε
void Ep() {
    if (expr[count] == '+') {
        count++;
        T();
        Ep();
    }
}

// T → check Tp
void T() {
    check();
    Tp();
}

// Tp → * check Tp | ε
void Tp() {
    if (expr[count] == '*') {
        count++;
        check();
        Tp();  // Recursive call inside the if-block
    }
}

// check → (E) | alphanumeric
void check() {
    if (isalnum(expr[count])) {
        count++;
    } else if (expr[count] == '(') {
        count++;
        E();
        if (expr[count] == ')') {
            count++;
        } else {
            flag = 1;  // Missing closing parenthesis
        }
    } else {
        flag = 1;  // Invalid character
    }
}


In [ ]:
#include <stdio.h>
#include <ctype.h>
#include <string.h>

char input[100];
int i = 0;

// Function declarations
void E(); void Eprime();
void T(); void Tprime();
void F();

void error() {
    printf("Syntax Error!\n");
    exit(1);
}

// E → T E'
void E() {
    T();
    Eprime();
}

// E' → + T E' | - T E' | ε
void Eprime() {
    if (input[i] == '+') {
        i++;
        T();
        Eprime();
    } else if (input[i] == '-') {
        i++;
        T();
        Eprime();
    }
    // else ε (do nothing)
}

// T → F T'
void T() {
    F();
    Tprime();
}

// T' → * F T' | / F T' | ε
void Tprime() {
    if (input[i] == '*') {
        i++;
        F();
        Tprime();
    } else if (input[i] == '/') {
        i++;
        F();
        Tprime();
    }
    // else ε (do nothing)
}

// F → (E) | digit
void F() {
    if (isdigit(input[i])) {
        i++;  // Accept digit
    } else if (input[i] == '(') {
        i++;
        E();
        if (input[i] == ')') {
            i++;
        } else {
            error();
        }
    } else {
        error();
    }
}

int main() {
    printf("Enter an arithmetic expression: ");
    scanf("%s", input);

    E();  // Start from E (Expression)

    if (input[i] == '\0') {
        printf("Expression is valid!\n");
    } else {
        error();
    }

    return 0;
}


# 6.1

In [ ]:
#include <stdio.h>

#define TOGETHER 8  // Number of operations done together

int main(void) {
    int i = 0, entries = 15;
    int repeat, left = 0;

    repeat = entries / TOGETHER;  // Full chunks of 8
    left = entries % TOGETHER;    // Remaining entries

    // Loop unrolling for TOGETHER (8) entries
    while (repeat--) {
        printf("process(%d)\n", i);
        printf("process(%d)\n", i + 1);
        printf("process(%d)\n", i + 2);
        printf("process(%d)\n", i + 3);
        printf("process(%d)\n", i + 4);
        printf("process(%d)\n", i + 5);
        printf("process(%d)\n", i + 6);
        printf("process(%d)\n", i + 7);
        i += TOGETHER;
    }

    // Handle remaining entries using switch-case fall-through
    switch (left) {
        case 7: printf("process(%d)\n", i + 6);
        case 6: printf("process(%d)\n", i + 5);
        case 5: printf("process(%d)\n", i + 4);
        case 4: printf("process(%d)\n", i + 3);
        case 3: printf("process(%d)\n", i + 2);
        case 2: printf("process(%d)\n", i + 1);
        case 1: printf("process(%d)\n", i);
        case 0: ; // Do nothing for zero
    }

    return 0;
}


# 6.2

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <ctype.h>

struct expr {
    char op, op1[10], op2[10], res[10];
    int flag; // 1 = remove this expression
};

struct expr e[10];
int n;

void change(int index, char *value) {
    for (int i = index + 1; i < n; i++) {
        if (strcmp(e[index].res, e[i].op1) == 0)
            strcpy(e[i].op1, value);
        if (strcmp(e[index].res, e[i].op2) == 0)
            strcpy(e[i].op2, value);
    }
}

void propagateConstants() {
    for (int i = 0; i < n; i++) {
        if (isdigit(e[i].op1[0]) && isdigit(e[i].op2[0])) {
            int a = atoi(e[i].op1);
            int b = atoi(e[i].op2);
            int result;

            switch (e[i].op) {
                case '+': result = a + b; break;
                case '-': result = a - b; break;
                case '*': result = a * b; break;
                case '/': result = a / b; break;
                default: continue;
            }

            char value[10];
            sprintf(value, "%d", result);
            e[i].flag = 1;
            change(i, value);
        }
    }
}

int main() {
    printf("Enter number of expressions: ");
    scanf("%d", &n);

    printf("Enter expressions in format: op op1 op2 res (e.g., + 2 3 x)\n");
    for (int i = 0; i < n; i++) {
        scanf(" %c %s %s %s", &e[i].op, e[i].op1, e[i].op2, e[i].res);
        e[i].flag = 0;
    }

    propagateConstants();

    printf("\nOptimized code:\n");
    for (int i = 0; i < n; i++) {
        if (!e[i].flag)
            printf("%c %s %s %s\n", e[i].op, e[i].op1, e[i].op2, e[i].res);
    }

    return 0;
}


In [ ]:
r